In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2

from nibabel.viewers import OrthoSlicer3D

from segment_anything import SamAutomaticMaskGenerator
import supervision as sv

import nibabel as nib
import imageio
import os

import torch
import torchmetrics
from torchmetrics.functional import dice

import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

import json

from segment_anything import SamPredictor, sam_model_registry

# Process data
would takes 1.5 min on PC.

In [2]:
img_path = './data/RawData/Training/img/'
img_filenames = sorted(os.listdir(img_path))
label_path = './data/RawData/Training/label/'
label_filenames = sorted(os.listdir(label_path))

bbox_coords = {}
ground_truth_mask = {}
data_count = 0
for img_num in range(0, 30):
    # read image and label
    image_nib = nib.load(img_path + img_filenames[img_num])
    image = image_nib.get_fdata().astype(np.uint8)
    x, y, z = image.shape
    label_nib = nib.load(label_path + label_filenames[img_num])
    label = label_nib.get_fdata().astype(np.uint8)
    
    for h in range(z):
        image0 = image[:, :, h]
        label0 = label[:, :, h]
        
        for i in range(1, 14):
            org_label = np.where(label0 == i, 1, 0)
            nonzero_indices = np.argwhere(org_label == 1)
            if len(nonzero_indices) < 10:   # if the number of non-zero indices is too small, skip
                continue
            x_min = np.min(nonzero_indices[:, 0])
            y_min = np.min(nonzero_indices[:, 1])
            x_max = np.max(nonzero_indices[:, 0])
            y_max = np.max(nonzero_indices[:, 1])
                
            bbox = np.array([[x_min-3, y_min-3, x_max+3, y_max+3]])
            
            bbox_coords[data_count] = bbox.tolist()
            ground_truth_mask[data_count] = org_label.tolist()
            data_count += 1
            


# Prepare for Fine Tuning

In [ ]:
sam_checkpoint = "./checkpoints/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

sam_model = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam_model.to(device=device)

: 

: 

In [ ]:
# We convert the input images into a format SAM's internal functions expect.
# Preprocess the images
from collections import defaultdict

import torch

from segment_anything.utils.transforms import ResizeLongestSide

transformed_data = defaultdict(dict)
for k in bbox_coords.keys():
  image = cv2.imread(f'scans/scans/{k}.png')
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
  transform = ResizeLongestSide(sam_model.image_encoder.img_size)
  input_image = transform.apply_image(image)
  input_image_torch = torch.as_tensor(input_image, device=device)
  transformed_image = input_image_torch.permute(2, 0, 1).contiguous()[None, :, :, :]
  
  input_image = sam_model.preprocess(transformed_image)
  original_image_size = image.shape[:2]
  input_size = tuple(transformed_image.shape[-2:])

  transformed_data[k]['image'] = input_image
  transformed_data[k]['input_size'] = input_size
  transformed_data[k]['original_image_size'] = original_image_size